In [1]:
#v5 is trained on screenshot of video 

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import os
import datetime

# ----------------------------
# Paths & Parameters
# ----------------------------
train_dir = "dataset/train"
val_dir = "dataset/val"
test_dir = "dataset/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = len(os.listdir(train_dir))  # automatically count classes
EPOCHS_INITIAL = 20  # initial Dense layers training
EPOCHS_FINE = 15     # fine-tuning conv layers

NEW_MODEL_NAME = "project_model.keras"

# ----------------------------
# Data Generators 
# ----------------------------
train_datagen = ImageDataGenerator(rescale=1./255)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ----------------------------
# Build Model
# ----------------------------
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze all conv layers initially
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile for initial training
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ----------------------------
# Callbacks
# ----------------------------
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    ModelCheckpoint(NEW_MODEL_NAME, monitor='val_loss', save_best_only=True),
    TensorBoard(log_dir=log_dir)
]

# ----------------------------
# Train only Dense layers first
# ----------------------------
history_initial = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_INITIAL,
    callbacks=callbacks,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# ----------------------------
# Fine-tune last few conv layers
# ----------------------------
# Unfreeze last 5 conv layers of VGG19
for layer in base_model.layers[-8:]:  # last block 5 conv layers
    layer.trainable = True

# Compile with smaller LR for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_FINE,



    
    callbacks=callbacks,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# ----------------------------
# Evaluate on test set
# ----------------------------
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")


Found 2501 images belonging to 15 classes.
Found 710 images belonging to 15 classes.
Found 370 images belonging to 15 classes.
Epoch 1/20
10/79 ━━━━━━━━━━━━━━━━━━━━ 10:39 9s/step - accuracy: 0.0624 - loss: 2.8649